In [11]:
import pandas as pd
import sqlite3
import os
import re

This is where we will clean the data and use it to create a database of all the data to facilitate its use.

First we need to read the country data into a dataframe.

In [3]:
df_country = pd.read_csv('./data/raw-data/arrivals/Metadata_Country_API_ST.INT.ARVL_DS2_en_csv_v2_5994899.csv')

#df_country.rename(columns={''country_code', 'region', 'income_group', 'special_notes', 'country', 'unnamed'}], inplace=True)
df_country.head(5)

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,ABW,Latin America & Caribbean,High income,NaN,Aruba,NaN
1,AFE,NaN,NaN,"26 countries, stretching from the Red Sea in t...",Africa Eastern and Southern,NaN
2,AFG,South Asia,Low income,The reporting period for national accounts dat...,Afghanistan,NaN
3,AFW,NaN,NaN,"22 countries, stretching from the westernmost ...",Africa Western and Central,NaN
4,AGO,Sub-Saharan Africa,Lower middle income,The World Bank systematically assesses the app...,Angola,NaN


In [4]:
df_country.rename(columns={'Country Code': 'country_code', 
                           'Region': 'region',
                           'IncomeGroup': 'income_group',
                           'SpecialNotes': 'special_notes',
                           'TableName': 'country',
                           'Unnamed: 5': 'unnamed'}, inplace=True)

df_country.head()

,country_code,region,income_group,special_notes,country,unnamed
0,ABW,Latin America & Caribbean,High income,NaN,Aruba,NaN
1,AFE,NaN,NaN,"26 countries, stretching from the Red Sea in t...",Africa Eastern and Southern,NaN
2,AFG,South Asia,Low income,The reporting period for national accounts dat...,Afghanistan,NaN
3,AFW,NaN,NaN,"22 countries, stretching from the westernmost ...",Africa Western and Central,NaN
4,AGO,Sub-Saharan Africa,Lower middle income,The World Bank systematically assesses the app...,Angola,NaN


The next bit of code determines if the mysterious 'unnamed' field contains any data.

In [5]:
df_country[df_country.unnamed.notnull()]

,country_code,region,income_group,special_notes,country,unnamed


Since it does not contain anything, we can safely remove it. We can also remove the "special_notes" since they're not very useful for our purpose.

In [6]:
df_country.drop(columns=['special_notes', 'unnamed'], inplace=True)

df_country.head()

,country_code,region,income_group,country
0,ABW,Latin America & Caribbean,High income,Aruba
1,AFE,NaN,NaN,Africa Eastern and Southern
2,AFG,South Asia,Low income,Afghanistan
3,AFW,NaN,NaN,Africa Western and Central
4,AGO,Sub-Saharan Africa,Lower middle income,Angola


Next, let's reorder the columns to make the data easier for us to read and mentally process.

In [7]:
df_country = df_country[['country_code', 'country', 'region', 'income_group']]
df_country.head()

,country_code,country,region,income_group
0,ABW,Aruba,Latin America & Caribbean,High income
1,AFE,Africa Eastern and Southern,NaN,NaN
2,AFG,Afghanistan,South Asia,Low income
3,AFW,Africa Western and Central,NaN,NaN
4,AGO,Angola,Sub-Saharan Africa,Lower middle income


So far, it's looking much more readable; however, there are also aggregate regions that need to be removed for clarity, considering they are also listed under 'region' in the rows which are for an actual country.

In [34]:
df_country.drop(axis=0, index=df_country.index[df_country.region.isnull()], inplace=True)
df_country.reset_index(drop=True, inplace=True)
df_country.head()


,country_code,country,region,income_group
0,ABW,Aruba,Latin America & Caribbean,High income
1,AFG,Afghanistan,South Asia,Low income
2,AGO,Angola,Sub-Saharan Africa,Lower middle income
3,ALB,Albania,Europe & Central Asia,Upper middle income
4,AND,Andorra,Europe & Central Asia,High income


Now that we've cleaned this up, let's export it to a SQL database for ease of use.

In [36]:
connection = sqlite3.connect('./data/db/tourism.db')

df_country.to_sql('country', connection, if_exists='replace')

217

One data source down! Now let's repeat the process for the others.

In [46]:
df_arrivals = pd.read_csv('./data/raw-data/arrivals/API_ST.INT.ARVL_DS2_en_csv_v2_5994899.csv')

df_arrivals.drop(axis=1, labels=['Indicator Name', 'Indicator Code', 'Unnamed: 67'], inplace=True)
df_arrivals.rename(columns={'Country Name': 'country_name', 'Country Code': 'country_code'}, inplace=True)
df_arrivals.head()


,country_name,country_code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.667000e+06,1.739000e+06,1.832000e+06,1.758000e+06,1.863000e+06,1.897000e+06,1.951000e+06,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.442663e+07,3.573839e+07,3.531868e+07,3.764589e+07,3.825835e+07,4.118915e+07,3.982670e+07,NaN,NaN,NaN
2,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.008582e+07,1.054462e+07,1.331168e+07,1.315078e+07,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.500000e+05,5.950000e+05,5.920000e+05,3.970000e+05,2.610000e+05,2.180000e+05,2.180000e+05,NaN,NaN,NaN


It looks like there are some columns with no entries whatsoever. Let's get rid of those.

In [47]:
df_arrivals.dropna(axis=1, how='all', inplace=True)

df_arrivals.head()

,country_name,country_code,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,9.120000e+05,9.570000e+05,9.470000e+05,9.060000e+05,9.720000e+05,1.211000e+06,1.178000e+06,1.225000e+06,...,1.469000e+06,1.481000e+06,1.667000e+06,1.739000e+06,1.832000e+06,1.758000e+06,1.863000e+06,1.897000e+06,1.951000e+06,NaN
1,Africa Eastern and Southern,AFE,1.158354e+07,1.308865e+07,1.345625e+07,1.440385e+07,1.530938e+07,1.535318e+07,1.585470e+07,1.738338e+07,...,3.165024e+07,3.274855e+07,3.442663e+07,3.573839e+07,3.531868e+07,3.764589e+07,3.825835e+07,4.118915e+07,3.982670e+07,NaN
2,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,2.670706e+06,3.027135e+06,3.243144e+06,3.422652e+06,3.897975e+06,4.162850e+06,4.615887e+06,4.697120e+06,...,8.902380e+06,1.022103e+07,1.008582e+07,1.054462e+07,1.331168e+07,1.315078e+07,NaN,NaN,NaN,NaN
4,Angola,AGO,9.000000e+03,2.100000e+04,4.500000e+04,5.200000e+04,4.500000e+04,5.100000e+04,6.700000e+04,9.100000e+04,...,4.810000e+05,5.280000e+05,6.500000e+05,5.950000e+05,5.920000e+05,3.970000e+05,2.610000e+05,2.180000e+05,2.180000e+05,NaN


Let's also move the 'country_code' column to the first row for consistency's sake.

In [51]:
arrival_columns = df_arrivals.columns.to_list()

arrival_columns[0] = 'country_code'
arrival_columns[1] = 'country_name'

df_arrivals = df_arrivals[arrival_columns]
df_arrivals.head()

,country_code,country_name,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,Aruba,9.120000e+05,9.570000e+05,9.470000e+05,9.060000e+05,9.720000e+05,1.211000e+06,1.178000e+06,1.225000e+06,...,1.469000e+06,1.481000e+06,1.667000e+06,1.739000e+06,1.832000e+06,1.758000e+06,1.863000e+06,1.897000e+06,1.951000e+06,NaN
1,AFE,Africa Eastern and Southern,1.158354e+07,1.308865e+07,1.345625e+07,1.440385e+07,1.530938e+07,1.535318e+07,1.585470e+07,1.738338e+07,...,3.165024e+07,3.274855e+07,3.442663e+07,3.573839e+07,3.531868e+07,3.764589e+07,3.825835e+07,4.118915e+07,3.982670e+07,NaN
2,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,Africa Western and Central,2.670706e+06,3.027135e+06,3.243144e+06,3.422652e+06,3.897975e+06,4.162850e+06,4.615887e+06,4.697120e+06,...,8.902380e+06,1.022103e+07,1.008582e+07,1.054462e+07,1.331168e+07,1.315078e+07,NaN,NaN,NaN,NaN
4,AGO,Angola,9.000000e+03,2.100000e+04,4.500000e+04,5.200000e+04,4.500000e+04,5.100000e+04,6.700000e+04,9.100000e+04,...,4.810000e+05,5.280000e+05,6.500000e+05,5.950000e+05,5.920000e+05,3.970000e+05,2.610000e+05,2.180000e+05,2.180000e+05,NaN


This also has the aggregate regions that we can do away with.

In [80]:
country_codes = df_country.country_code.to_list()

df_arrivals = df_arrivals[df_arrivals.country_code.isin(country_codes)]

df_arrivals.reset_index(drop=True, inplace=True)

df_arrivals.head()

,country_code,country_name,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,Aruba,912000.0,957000.0,947000.0,906000.0,972000.0,1211000.0,1178000.0,1225000.0,...,1469000.0,1481000.0,1667000.0,1739000.0,1832000.0,1758000.0,1863000.0,1897000.0,1951000.0,NaN
1,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AGO,Angola,9000.0,21000.0,45000.0,52000.0,45000.0,51000.0,67000.0,91000.0,...,481000.0,528000.0,650000.0,595000.0,592000.0,397000.0,261000.0,218000.0,218000.0,NaN
3,ALB,Albania,304000.0,287000.0,119000.0,184000.0,371000.0,317000.0,354000.0,470000.0,...,2932000.0,3514000.0,3256000.0,3673000.0,4131000.0,4736000.0,5118000.0,5927000.0,6406000.0,2658000.0
4,AND,Andorra,NaN,NaN,NaN,NaN,9422000.0,10991000.0,11351000.0,11507000.0,...,7983000.0,7900000.0,7676000.0,7797000.0,7850000.0,8025000.0,8152000.0,8328000.0,8235000.0,5207000.0


Now let's get this one saved as a database as well.

In [81]:
df_arrivals.to_sql('arrivals', connection, if_exists='replace')

217

Continuing on...

In [99]:
df_departures = pd.read_csv('./data/raw-data/departures/API_ST.INT.DPRT_DS2_en_csv_v2_5996775.csv')
df_expenditures = pd.read_csv('./data/raw-data/expenditures/API_ST.INT.XPND.CD_DS2_en_csv_v2_5996767.csv')
df_gdp = pd.read_csv('./data/raw-data/gdp/API_NY.GDP.MKTP.KD_DS2_en_csv_v2_5994841.csv')
df_gdp_per_capita = pd.read_csv('./data/raw-data/gdp-per-capita/API_NY.GDP.PCAP.KD_DS2_en_csv_v2_5994684.csv')
df_income = pd.read_csv('./data/raw-data/income/API_NY.ADJ.NNTY.PC.CD_DS2_en_csv_v2_5996043.csv')
df_receipts = pd.read_csv('./data/raw-data/receipts/API_ST.INT.RCPT.CD_DS2_en_csv_v2_5996774.csv')

In [100]:
df_departures.drop(columns=['Indicator Name', 'Indicator Code'], inplace=True)
df_departures.dropna(axis=1, how='all', inplace=True)
df_departures.rename(columns={'Country Name': 'country_name', 'Country Code': 'country_code'}, inplace=True)
departure_columns = df_departures.columns.to_list()
departure_columns[0] = 'country_code'
departure_columns[1] = 'country_name'
df_departures = df_departures[departure_columns]
df_departures = df_departures[df_departures.country_code.isin(country_codes)]
df_departures.reset_index(drop=True, inplace=True)

df_departures.head()


,country_code,country_name,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AGO,Angola,3000.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALB,Albania,NaN,NaN,NaN,NaN,NaN,NaN,955000.0,1303000.0,...,4120000.0,3959000.0,3928000.0,4146000.0,4504000.0,4852000.0,5186000.0,5415000.0,5922000.0,2907000.0
4,AND,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
df_departures.to_sql('departures', connection, if_exists='replace')

217

In [102]:
df_expenditures.drop(columns=['Indicator Name', 'Indicator Code'], inplace=True)
df_expenditures.dropna(axis=1, how='all', inplace=True)
df_expenditures.rename(columns={'Country Name': 'country_name', 'Country Code': 'country_code'}, inplace=True)
expenditure_columns = df_expenditures.columns.to_list()
expenditure_columns[0] = 'country_code'
expenditure_columns[1] = 'country_name'
df_expenditures = df_expenditures[expenditure_columns]
df_expenditures = df_expenditures[df_expenditures.country_code.isin(country_codes)]
df_expenditures.reset_index(drop=True, inplace=True)
df_expenditures.head()

,country_code,country_name,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,Aruba,79000000.0,128000000.0,161000000.0,140000000.0,151000000.0,163000000.0,156000000.0,172000000.0,...,2.870000e+08,2.940000e+08,3.420000e+08,3.500000e+08,3.570000e+08,3.190000e+08,3.490000e+08,3.940000e+08,3.990000e+08,310000000.0
1,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.550000e+08,1.100000e+08,1.380000e+08,1.400000e+08,1.510000e+08,8.900000e+07,1.300000e+08,2.260000e+08,1.680000e+08,49000000.0
2,AGO,Angola,113000000.0,110000000.0,125000000.0,96000000.0,142000000.0,146000000.0,80000000.0,52000000.0,...,3.230000e+08,2.880000e+08,3.160000e+08,5.050000e+08,3.890000e+08,8.230000e+08,1.216000e+09,7.620000e+08,7.170000e+08,691000000.0
3,ALB,Albania,19000000.0,25000000.0,13000000.0,22000000.0,35000000.0,290000000.0,269000000.0,386000000.0,...,1.677000e+09,1.374000e+09,1.567000e+09,1.689000e+09,1.311000e+09,1.338000e+09,1.473000e+09,1.750000e+09,1.852000e+09,805000000.0
4,AND,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.870000e+08,NaN


In [103]:
df_expenditures.to_sql('expenditures', connection, if_exists='replace')

217

In [104]:
df_gdp.drop(columns=['Indicator Name', 'Indicator Code'], inplace=True)
df_gdp.dropna(axis=1, how='all', inplace=True)
df_gdp.rename(columns={'Country Name': 'country_name', 'Country Code': 'country_code'}, inplace=True)
gdp_columns = df_gdp.columns.to_list()
gdp_columns[0] = 'country_code'
gdp_columns[1] = 'country_name'
df_gdp = df_gdp[gdp_columns]
df_gdp = df_gdp[df_gdp.country_code.isin(country_codes)]
df_gdp.reset_index(drop=True, inplace=True)
df_gdp.head()

,country_code,country_name,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ABW,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.862306e+09,2.861720e+09,2.963128e+09,3.025850e+09,3.191738e+09,3.359555e+09,3.380889e+09,2.752412e+09,3.225070e+09,NaN
1,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.918924e+10,1.971206e+10,1.999814e+10,2.045016e+10,2.099148e+10,2.124112e+10,2.207199e+10,2.155305e+10,1.709157e+10,NaN
2,AGO,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.242884e+10,8.640402e+10,8.721930e+10,8.496895e+10,8.484391e+10,8.372706e+10,8.313907e+10,7.845151e+10,7.939231e+10,8.181012e+10
3,ALB,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.094547e+10,1.113969e+10,1.138685e+10,1.176433e+10,1.221168e+10,1.270250e+10,1.296770e+10,1.253949e+10,1.365658e+10,1.431813e+10
4,AND,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.683235e+09,2.750436e+09,2.789881e+09,2.893377e+09,2.903390e+09,2.949518e+09,3.008967e+09,2.672446e+09,2.893917e+09,3.148859e+09


In [105]:
df_gdp.to_sql('gdp', connection, if_exists='replace')

217

In [111]:
df_gdp_per_capita.drop(columns=['Indicator Name', 'Indicator Code'], inplace=True)
df_gdp_per_capita.dropna(axis=1, how='all', inplace=True)
df_gdp_per_capita.rename(columns={'Country Name': 'country_name', 'Country Code': 'country_code'}, inplace=True)
gdp_columns = df_gdp_per_capita.columns.to_list()
gdp_columns[0] = 'country_code'
gdp_columns[1] = 'country_name'
df_gdp_per_capita = df_gdp_per_capita[gdp_columns]
df_gdp_per_capita = df_gdp_per_capita[df_gdp_per_capita.country_code.isin(country_codes)]
df_gdp_per_capita.reset_index(drop=True, inplace=True)
df_gdp_per_capita.head()

KeyError: "['Indicator Name', 'Indicator Code'] not found in axis"

In [112]:
df_gdp_per_capita.to_sql('gdp_per_capita', connection, if_exists='replace')

217

In [113]:
df_income.drop(columns=['Indicator Name', 'Indicator Code'], inplace=True)
df_income.dropna(axis=1, how='all', inplace=True)
df_income.rename(columns={'Country Name': 'country_name', 'Country Code': 'country_code'}, inplace=True)
income_columns = df_income.columns.to_list()
income_columns[0] = 'country_code'
income_columns[1] = 'country_name'
df_income = df_income[income_columns]
df_income = df_income[df_income.country_code.isin(country_codes)]
df_income.reset_index(drop=True, inplace=True)
df_income.head()

,country_code,country_name,1970,1971,1972,1973,1974,1975,1976,1977,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20090.351473,21165.636596,22168.224174,23163.637761,24192.616602,24118.031802,24565.934645,24994.147351,NaN,NaN
1,AFG,Afghanistan,154.284558,157.65087,134.379044,141.664823,171.38238,182.45692,192.466274,218.010807,...,556.163841,598.215245,586.613689,575.680796,526.930235,489.009740,494.934920,458.344566,467.186947,475.084737
2,AGO,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2379.084597,2835.320063,3158.544114,3583.800549,3199.012702,2688.630467,2936.850718,2051.814728,1738.059253,1103.908798
3,ALB,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3728.047790,3503.243510,3760.376907,3844.337544,3267.217515,3443.310224,3712.733017,4318.911492,4337.131083,4208.456611
4,AND,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
df_income.to_sql('income', connection, if_exists='replace')

217

In [115]:
df_receipts.drop(columns=['Indicator Name', 'Indicator Code'], inplace=True)
df_receipts.dropna(axis=1, how='all', inplace=True)
df_receipts.rename(columns={'Country Name': 'country_name', 'Country Code': 'country_code'}, inplace=True)
receipts_columns = df_receipts.columns.to_list()
receipts_columns[0] = 'country_code'
receipts_columns[1] = 'country_name'
df_receipts = df_receipts[receipts_columns]
df_receipts = df_receipts[df_receipts.country_code.isin(country_codes)]
df_receipts.reset_index(drop=True, inplace=True)
df_receipts.head()

,country_code,country_name,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,Aruba,554000000.0,666000000.0,726000000.0,786000000.0,782000000.0,850000000.0,825000000.0,835000000.0,...,1.358000e+09,1.412000e+09,1.506000e+09,1.625000e+09,1.659000e+09,1.757000e+09,1.855000e+09,2.035000e+09,2.109000e+09,1.077000e+09
1,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.650000e+08,1.670000e+08,1.790000e+08,1.210000e+08,8.600000e+07,6.200000e+07,1.600000e+07,5.000000e+07,8.500000e+07,7.500000e+07
2,AGO,Angola,27000000.0,38000000.0,24000000.0,39000000.0,31000000.0,34000000.0,35000000.0,51000000.0,...,6.530000e+08,7.110000e+08,1.241000e+09,1.597000e+09,1.171000e+09,6.280000e+08,8.840000e+08,5.570000e+08,3.950000e+08,1.900000e+07
3,ALB,Albania,70000000.0,94000000.0,34000000.0,60000000.0,218000000.0,398000000.0,451000000.0,492000000.0,...,1.833000e+09,1.623000e+09,1.670000e+09,1.849000e+09,1.613000e+09,1.821000e+09,2.050000e+09,2.306000e+09,2.458000e+09,1.243000e+09
4,AND,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.910000e+09,NaN


In [116]:
df_receipts.to_sql('receipts', connection, if_exists='replace')

217